In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
import os
import time
import datetime
from google.colab import drive
import pickle
import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)
import itertools

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
!nvidia-smi

In [ ]:
import transforms as tf
import generators as gens
from generators import from_free_group, from_normal_closure, uniform_hyperbolic_length
from free_group import is_from_normal_closure

def check_in_closer(generated_words):
  alg_words = []
  for generated_word in generated_words:
    output = tokenizer.decode(generated_word, skip_special_tokens=True)
    output1 = (list(map(int, [i for i in [i for i in output]])))
    output1 = list(map(int, [words[x] if words else x for x in output1]))
    if (is_from_normal_closure([1], output1) == True):
      alg_words.append(output1)
      #print(word_as_str(output1))
  print(len(alg_words)*100/len(generated_words), '%')

def word_as_str(word):
    letters = "xyzpqrstuvwklme"
    return "".join(map(lambda factor: letters[abs(factor) - 1] + ("⁻¹" if factor < 0 else ""), word))

def print_word(word_):
  a = (list(map(int, [i for i in [i for i in tokenizer.decode(word_, skip_special_tokens=True)]])))
  print(word_as_str(list(map(int, [words[x] if words else x for x in a]))))

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') #gpt2-medium

In [ ]:
import json
f = open('/content/drive/MyDrive/Folder/datasets/_xyzp_-10.json')
data = json.load(f)
f.close()

In [ ]:
def find_max_list(list):
    list_len = [len(i) for i in list]
    print(max(list_len))

find_max_list(data)

In [ ]:
sentenses = []
for element in data_train:
  results = list(map(str, element['data']))
  if element['label'][0] == 1:
    sentenses.append(results)

In [ ]:
sentenses = data
merged = list(itertools.chain(*sentenses))
words = list(set(merged))  #коллекция уникальных элементов
word2idx = dict((word, i) for i, word in enumerate(words))
token_stream_data = []
for i in sentenses:
  C = (pd.Series(i)).map(word2idx) 
  token_stream_data.append(list(C))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  import sys


In [ ]:
def tokenization(token_data):
  token_seq = []
  for xs in token_data:     
    a = "".join(str(x) for x in xs)
    token_seq.append(a)
  return token_seq

In [ ]:
train_texts = tokenization(token_stream_data)

In [ ]:
class GPT2Dataset(Dataset):
  def __init__(self, txt_list, tokenizer, gpt2_type="gpt2", max_length=768):
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    for txt in txt_list:
      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', max_length=max_length, padding="max_length", truncation=True)
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx] 

In [ ]:
batch_size = 14
max_length = 160

In [ ]:
dataset = GPT2Dataset(train_texts, tokenizer, max_length=max_length)#768)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
train_dataloader = DataLoader( train_dataset,   sampler = RandomSampler(train_dataset),   batch_size = batch_size)
validation_dataloader = DataLoader(  val_dataset,   sampler = SequentialSampler(val_dataset),  batch_size = batch_size)

In [ ]:
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model1 = GPT2LMHeadModel.from_pretrained('gpt2', config=configuration)
model1.resize_token_embeddings(len(tokenizer))

In [ ]:
model = model1
device = torch.device("cuda")
model.cuda()
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
epochs = 6
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8
sample_every = 200
optimizer = AdamW(model.parameters(),  lr = learning_rate,   eps = epsilon )

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,   num_warmup_steps = warmup_steps, num_training_steps = total_steps)
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()
training_stats = []
model = model.to(device)
for epoch_i in range(0, epochs):
    #               Training
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')
    t0 = time.time()

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        #tokenizer.pad_token_id = tokenizer.eos_token_id

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)

        #b_labels[b_labels == 2598] = -100
        #bool_mask_invert = ~bool_mask.bool()
        #b_labels[bool_mask_invert] = -100
        b_masks = batch[1].to(device)

        model.zero_grad()        
        
        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None )
        loss = outputs[0]  
        batch_loss = loss.item()
        total_train_loss += batch_loss
        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()
            prompt = "<|startoftext|>"
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
            generated = generated.to(device)
            sample_outputs = model.generate(generated,
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=200,
                                     #pad_token_id = 50258
                                      )

            
            #performance_estimate_inters(sample_outputs)
            
            a = []
            for generated_word in sample_outputs:
                output1 = [1,2,3]
                try:
                  output = tokenizer.decode(generated_word, skip_special_tokens=True)
                  output1 = (list(map(int, [i for i in [i for i in output]])))
                  output1 = list(map(int, [words[x] if words else x for x in output1]))
                except: 
                  print('error')
                  pass
                a.append(output1)
            eavluate_sampled(a, 5)
            for i, sample_output in enumerate(sample_outputs[:3]):
                #print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
                #print(sample_output)
                pass
                #output = tokenizer.decode(sample_output, skip_special_tokens=True)
                #print(output)
                #output1 = (list(map(int, [i for i in [i for i in output]])))
                #output1 = list(map(int, [words[x] if words else x for x in output1]))
                #print(word_as_str(output1))

            model.train()

        loss.backward()
        optimizer.step()
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))       
    #               Validation
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        #b_labels[b_labels == 2816] = -100
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
                          #  token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
          { 'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time})
    
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
configuration = GPT2Config.from_pretrained('/content/drive/MyDrive/Folder/model_gpt2_<xyzp>_130/config.json', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Folder/model_gpt2_<xyzp>_130/pytorch_model.bin', config=configuration)
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/Folder/model_gpt2_<xyzp>_130', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>') 

In [ ]:
from datetime import datetime
import time

start_time = datetime.now()
device = torch.device("cuda")
#model.cuda()
model = model.to(device)
model.eval()
prompt = "<|startoftext|>"
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)
examples_number = 600
sample_outputs = model.generate(generated, do_sample=True, top_k=10, max_length = 100, 
                                top_p=0.95, num_return_sequences= examples_number)
print(datetime.now() - start_time)

In [ ]:
a = []
for generated_word in sample_outputs:
    output = tokenizer.decode(generated_word, skip_special_tokens=True)
    output1 = (list(map(int, [i for i in [i for i in output]])))
    output1 = list(map(int, [words[x] if words else x for x in output1]))
    output1 = normalize(output1)
    a.append(output1)

alg4, alg4_ = eavluate_sampled(a, 4)

In [ ]:
def word_as_str(word) -> str:
    letters = "xyzpqrstuvwklmn"
    inverse_letters = letters.upper()
    return ''.join([letters[abs(f) - 1] if f >= 0 else inverse_letters[abs(f) - 1] for f in word])

def eavluate_sampled(sampled: list, n_generators: int):
    bases   = [[i] for i in range(1, n_generators + 1)] + [list(range(1, n_generators + 1))]
    uniques = [set() for _ in range(n_generators + 1)]

    length_threshold = 20

    for seq in sampled:
        word = word_as_str(seq)
        for base, unique in zip(bases, uniques):
            #unique = list(unique)
            for t in range(length_threshold, len(word)):
                if is_from_normal_closure(base, seq[:t+1]):# and (seq[:t+1] not in data):
                    unique.add(word[:(t+1)])

    a = []
    for i in uniques:
      a.append(set.intersection(i))
    print('1',len(set.union(*a)))  
    print('2',len(set.union(set.intersection(*uniques[0:2]), set.intersection(*uniques[1:3]), set.intersection(uniques[0], uniques[2]))))
    print('symmetric commutant',len(set.intersection(*uniques[:3])))
    print('full',len(set.intersection(*uniques[:])))
    #print(set.intersection(*uniques[:]))
    return set.intersection(*uniques[:4]), set.intersection(*uniques[:])

def normalize(word):
    normalized = []

    for factor in word:
        if factor == 0:
            continue
        if len(normalized) == 0:
            normalized.append(factor)
            continue

        if factor == -normalized[-1]:
            normalized.pop()
        else:
            normalized.append(factor)

    return normalized